<a href="https://colab.research.google.com/github/ElBenjaM/Colab/blob/main/informes_Jw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Configuración (Obligatorio)
import time
from ipywidgets import Button, VBox, Text
from openpyxl.styles import Alignment
from IPython.display import clear_output, Markdown
from google.colab import drive
drive.mount('/content/drive')
!mkdir "/content/drive/MyDrive/informes_predicacion"
import openpyxl
clear_output()
import os
meta_horas=0
# La ruta a la que queremos acceder
ruta = '/content/drive/MyDrive/informes_predicacion/'

# El nombre del archivo que queremos buscar
nombre_archivo = 'informe_predicacion_marzo.xlsx'

# Comprobamos si existe el archivo en la ruta especificada
if os.path.exists(os.path.join(ruta, nombre_archivo)):
  # Si el archivo existe, mostramos un mensaje
  print("Listo!")
else:
  display(Markdown("**( Se puede dejar vacio - Apretar Guardar )**"))
  # Crea una caja de texto para ingresar la meta de horas
  meta_horas_text = Text(placeholder='Meta de horas mes de Marzo')

  # Crea un botón para guardar la meta de horas
  guardar_button = Button(description='Guardar')

  # Crea un contenedor vertical para mostrar los widgets
  container = VBox([meta_horas_text, guardar_button])

  # Muestra el contenedor
  display(container)

  # Define una función para manejar el evento de clic del botón
  def on_guardar_clicked(b):
  #HOJA 1
      meta_horas = int(meta_horas_text.value)
      workbook = openpyxl.Workbook()
      sheet = workbook.create_sheet("Registros")
      workbook.remove(workbook['Sheet'])

      sheet['A1'] = 'Fecha'
      sheet['B1'] = 'Horario'
      sheet['C1'] = 'Horas predicadas'
      sheet['D1'] = 'Minutos predicados'
      sheet['E1'] = 'Publicaciones'
      sheet['F1'] = 'Videos'
      sheet['G1'] = 'Revisitas' 
      sheet['H1'] = 'Notas'  
        
        # Centrar el texto y ajustar el ancho de las columnas
      sheet.column_dimensions['A'].width = 12
      sheet.column_dimensions['B'].width = 12
      sheet.column_dimensions['C'].width = 18
      sheet.column_dimensions['D'].width = 18
      sheet.column_dimensions['E'].width = 14
      sheet.column_dimensions['F'].width = 10
      sheet.column_dimensions['G'].width = 12
      sheet.column_dimensions['H'].width = 35

      # Centrar el texto y ajustar el ancho de las columnas
      for col in ['A', 'B', 'C', 'D', 'E', "F", "G", "H"]:
          column_dimension = sheet.column_dimensions[col]
          for cell in sheet[col]:
              alignment = Alignment(horizontal='center')
              cell.alignment = alignment

    # Crear una segunda hoja y escribir en las celdas y usar funciones de Excel
      sheet_totales = workbook.create_sheet("Totales")
      sheet_totales['A1'] = 'Total Horas'
      sheet_totales['B1'] = 'Total Minutos'
      sheet_totales['C1'] = 'Total Publicaciones'
      sheet_totales['D1'] = 'Total Videos'
      sheet_totales['E1'] = 'Total Revisitas'

      
      if meta_horas > 0:
          sheet_totales['A3'] = 'Meta Horas: '+str(meta_horas)


      # Centrar el texto hoja 2
      for col in ['A', 'B', 'C', 'D', 'E', 'F']:
          for cell in sheet_totales[col]:
              alignment = Alignment(horizontal='center')
              cell.alignment = alignment

      # Centrar el texto y ajustar el ancho de las columnas

      sheet_totales.column_dimensions['A'].width = 16
      sheet_totales.column_dimensions['B'].width = 16
      sheet_totales.column_dimensions['C'].width = 18
      sheet_totales.column_dimensions['D'].width = 16
      sheet_totales.column_dimensions['E'].width = 16


      # Guarda el libro de trabajo con el nombre deseado
      workbook.save("/content/drive/MyDrive/informes_predicacion/informe_predicacion_marzo.xlsx")
      clear_output()
      print("Se creó el Excel en tu Google Drive! (/My Drive/informes_predicacion/informe_predicacion_marzo.xlsx)")

  # Asigna la función manejadora al evento de clic del botón
  guardar_button.on_click(on_guardar_clicked)

In [ ]:
#@title Ingresar datos 
# Importa la librería y los módulos 
import numpy as np
from scipy.io.wavfile import write
from IPython.display import clear_output
from ipywidgets import Button, HBox, VBox, Text, Dropdown, DatePicker
import datetime
import openpyxl
from openpyxl.styles import Alignment
from openpyxl.utils import get_column_letter
from openpyxl.worksheet.dimensions import ColumnDimension
clear_output()
nombre_excel = "informe_predicacion.xlsx"
total_minutos = 0
total_horas_xd = 0
Total_Publicaciones = 0
Total_Videos  = 0
Total_Revisitas = 0
# Crea una caja de texto para ingresar las Horas Predicadas
horas_predi= Text(placeholder='Horas Predicadas')
# Crea una caja de texto para ingresar los minutos predicados
minutos_predi = Text(placeholder='Minutos Predicados')
# Crea una caja de texto para ingresar las Publicaciones
Publicaciones = Text(placeholder='Publicaciones entregadas')
# Crea una caja de texto para ingresar los Videos
Videos = Text(placeholder='Videos mostrados')
# Crea una caja de texto para ingresar las Revisitas 
Revisitas = Text(placeholder='Revisitas')
# Crea una caja de texto para ingresar la notas
notas = Text(placeholder='Notas')

# Crea el widget de selección de fechas y especifica el formato de la fecha
calendario = DatePicker()
display(calendario)  # Muestra el calendario

# Crea una celda de menú desplegable para seleccionar el tipo de predicación
tipo_predicacion_menu = Dropdown(options=["Predicación", "Carrito", "Estudio"])

# Crea un botón para descargar el audio
download_button = Button(description='Guardar')
XD = False
def on_button_clicked(b):
  global horas_predi, XD
  fecha_elegida = calendario.value
  if fecha_elegida is None:
    fecha_elegida = datetime.datetime.now().strftime('%d/%m/%Y')
  else:
    fecha_elegida = fecha_elegida.strftime('%d/%m/%Y')


  # Abrir el archivo de Excel existente
  workbook = openpyxl.load_workbook('/content/drive/MyDrive/informes_predicacion/informe_predicacion_marzo.xlsx')
  sheet = workbook.active

  #Determinar la siguiente fila disponible para escribir datos
  last_row = sheet.max_row + 1

  # Determinar la última fila con datos en el archivo de Excel
  #last_row = 1
  #for row in sheet.iter_rows():
    #last_row += 1
 

  # Escribir la fecha, la hora, las horas y minutos de predicación y la nota actuales en la siguiente fila
  sheet['A'+str(last_row)] = fecha_elegida
  sheet['B'+str(last_row)] = tipo_predicacion_menu.value

  if horas_predi.value == "":
    horas_predi.value = "0"
    sheet['C'+str(last_row)] = int(horas_predi.value)
  else: 
     sheet['C'+str(last_row)] = int(horas_predi.value)

  if minutos_predi.value == "":
    minutos_predi.value = "0"
    sheet['D'+str(last_row)] = int(minutos_predi.value)
  else: 
    sheet['D'+str(last_row)] = int(minutos_predi.value)

  if Publicaciones.value == "":
    Publicaciones.value = "0"
    sheet['E'+str(last_row)] = int(Publicaciones.value)
  else: 
    sheet['E'+str(last_row)] = int(Publicaciones.value)

  if Videos.value == "":
    Videos.value = "0"
    sheet['F'+str(last_row)] = int(Videos.value)
  else: 
    sheet['F'+str(last_row)] = int(Videos.value)

  if Revisitas.value == "":
    Revisitas.value = "0"
    sheet['G'+str(last_row)] = int(Revisitas.value)
  else: 
    sheet['G'+str(last_row)] = int(Revisitas.value)   

  if notas.value == "":
    notas.value = "Sin Nota"
    sheet['H'+str(last_row)] = notas.value
  else: 
    sheet['H'+str(last_row)] = notas.value
  
  
  # Guardar el libro de trabajo
  workbook.save('/content/drive/MyDrive/informes_predicacion/informe_predicacion_marzo.xlsx')
  clear_output()
  XD=True
  print("Listo!")

  if XD:
    global total_minutos , total_horas_xd, Total_Publicaciones, Total_Videos, Total_Revisitas
    # Abrir el archivo de Excel y seleccionar la hoja de trabajo
    workbook = openpyxl.load_workbook('/content/drive/MyDrive/informes_predicacion/informe_predicacion_marzo.xlsx')
    sheet = workbook["Registros"]

    # Inicializar la variable suma
    suma_horas = 0
    suma_minutos = 0
    total_minutos = 0
    total_horas_xd = 0
    Total_Publicaciones = 0
    Total_Videos  = 0
    Total_Revisitas = 0

    # Iterar a través de las filas y sumar los valores en la columna C
    for row in sheet.iter_rows(min_row=2, max_row=81, min_col=3, max_col=3):
      for cell in row:
        if cell.value is not None:
          suma_horas += cell.value
        else:
          suma_horas += 0


    # Iterar a través de las filas y sumar los valores en la columna C
    for row in sheet.iter_rows(min_row=2, max_row=81, min_col=4, max_col=4):
      for cell in row:
        if cell.value is not None:
          suma_minutos += cell.value
        else:
          suma_minutos += 0

    # Iterar a través de las filas y sumar los valores en la columna C
    for row in sheet.iter_rows(min_row=2, max_row=81, min_col=5, max_col=5):
      for cell in row:
        if cell.value is not None:
          Total_Publicaciones += cell.value
        else:
          Total_Publicaciones += 0

    # Iterar a través de las filas y sumar los valores en la columna C
    for row in sheet.iter_rows(min_row=2, max_row=81, min_col=6, max_col=6):
      for cell in row:
        if cell.value is not None:
          Total_Videos += cell.value
        else:
          Total_Videos += 0

    # Iterar a través de las filas y sumar los valores en la columna C
    for row in sheet.iter_rows(min_row=2, max_row=81, min_col=7, max_col=7):
      for cell in row:
        if cell.value is not None:
          Total_Revisitas += cell.value
        else:
          Total_Revisitas += 0                    



    total_horas_xd = suma_horas + int(suma_minutos/60)
    total_minutos = suma_minutos%60

    #Abrir hoja totales
    sheet_totales = workbook["Totales"]
    sheet_totales['A2'] = total_horas_xd
    sheet_totales['B2'] = total_minutos
    sheet_totales['C2'] = Total_Publicaciones
    sheet_totales['D2'] = Total_Videos
    sheet_totales['E2'] = Total_Revisitas

    # Leer el contenido de la celda A3 y guardar solo las últimas dos letras
    meta_horas = sheet_totales['A3'].value
    if meta_horas is not None:
        meta_horas = int(meta_horas[-2:])
        
    else:
        meta_horas = 0


    if meta_horas > 0:
      meta_tiempo = meta_horas
      #meta_tiempo = 23
      meta_tiempo = meta_tiempo-total_horas_xd
      #meta_tiempo = 23 -2 
      if total_minutos != 0:
        minutos_faltantes= 60-total_minutos
        if  meta_tiempo == 1 and minutos_faltantes >0:
            meta_tiempo = meta_tiempo - 1
      else:
          minutos_faltantes=0
          if  meta_tiempo == 1 and minutos_faltantes >0:
            meta_tiempo = meta_tiempo - 1
      if meta_tiempo > 1:
        if minutos_faltantes == 0:
              meta_tiempo = meta_tiempo
              sheet_totales['B3'] = "Te faltan: "+str(meta_tiempo)+" Horas"
        else:
              meta_tiempo = meta_tiempo - 1
              sheet_totales['B3'] = "Te faltan: "+str(meta_tiempo)+" Horas y "+str(minutos_faltantes)+" Minutos"
      if meta_tiempo == 1:
          if minutos_faltantes == 0:
              sheet_totales['B3'] = "Te falta una Hora"
          else: 
              
              sheet_totales['B3'] = "Te falta una Hora y "+str(minutos_faltantes)+" minutos."

      if meta_tiempo == 0:
        if minutos_faltantes == 0: 
              sheet_totales['B3'] ="Completaste tu Meta" 

        else:   
              sheet_totales['B3'] = "Te faltan: "+str(minutos_faltantes)+" Minutos"
          
      if meta_tiempo < 0:
      
              sheet_totales['B3'] ="Completaste tu Meta" 

          
    # Guarda el libro de trabajo con el nombre deseado
    workbook.save('/content/drive/MyDrive/informes_predicacion/informe_predicacion_marzo.xlsx')


# Asigna la función al evento "on_click" del botón
download_button.on_click(on_button_clicked)

# Muestra la caja de texto y el botón en el notebook
# Crea una caja vertical para mostrar todos los elementos de la interfaz de usuario
vbox = VBox([tipo_predicacion_menu,horas_predi, minutos_predi, Publicaciones, Videos,Revisitas, notas, download_button])

# Muestra la caja vertical en el notebook
display(vbox)


#------------------------------------------------------------------------------------------------

In [ ]:
#@title Ver Excel 
from ipywidgets import Button, HBox
from IPython.display import clear_output, Markdown
import pandas as pd
from google.colab import data_table
workbook = openpyxl.load_workbook('/content/drive/MyDrive/informes_predicacion/informe_predicacion_marzo.xlsx')
sheet = workbook["Registros"]

# Inicializar la variable suma
suma_horas = 0
suma_minutos = 0
total_minutos = 0
total_horas_xd = 0
Total_Publicaciones = 0
Total_Videos  = 0
Total_Revisitas = 0

# Iterar a través de las filas y sumar los valores en la columna C
for row in sheet.iter_rows(min_row=2, max_row=81, min_col=3, max_col=3):
  for cell in row:
    if cell.value is not None:
      suma_horas += cell.value
    else:
       suma_horas += 0


# Iterar a través de las filas y sumar los valores en la columna C
for row in sheet.iter_rows(min_row=2, max_row=81, min_col=4, max_col=4):
  for cell in row:
    if cell.value is not None:
      suma_minutos += cell.value
    else:
      suma_minutos += 0

# Iterar a través de las filas y sumar los valores en la columna C
for row in sheet.iter_rows(min_row=2, max_row=81, min_col=5, max_col=5):
  for cell in row:
    if cell.value is not None:
      Total_Publicaciones += cell.value
    else:
      Total_Publicaciones += 0

# Iterar a través de las filas y sumar los valores en la columna C
for row in sheet.iter_rows(min_row=2, max_row=81, min_col=6, max_col=6):
  for cell in row:
    if cell.value is not None:
      Total_Videos += cell.value
    else:
      Total_Videos += 0

# Iterar a través de las filas y sumar los valores en la columna C
for row in sheet.iter_rows(min_row=2, max_row=81, min_col=7, max_col=7):
  for cell in row:
    if cell.value is not None:
      Total_Revisitas += cell.value
    else:
      Total_Revisitas += 0                    



total_horas_xd = suma_horas + int(suma_minutos/60)
total_minutos = suma_minutos%60

#Abrir hoja totales
sheet_totales = workbook["Totales"]
sheet_totales['A2'] = total_horas_xd
sheet_totales['B2'] = total_minutos
sheet_totales['C2'] = Total_Publicaciones
sheet_totales['D2'] = Total_Videos
sheet_totales['E2'] = Total_Revisitas

# Leer el contenido de la celda A3 y guardar solo las últimas dos letras
meta_horas = sheet_totales['A3'].value
if meta_horas is not None:
    meta_horas = int(meta_horas[-2:])
    
else:
    meta_horas = 0


if meta_horas > 0:
  meta_tiempo = meta_horas
  #meta_tiempo = 25
  meta_tiempo = meta_tiempo-total_horas_xd
  #meta_tiempo = 25 - 24

  
  if total_minutos != 0:
        minutos_faltantes= 60-total_minutos
        if  meta_tiempo == 1 and minutos_faltantes >0:
            meta_tiempo = meta_tiempo - 1
  else:
          minutos_faltantes=0
          if  meta_tiempo == 1 and minutos_faltantes >0:
            meta_tiempo = meta_tiempo - 1
  if meta_tiempo > 1:
    if minutos_faltantes == 0:
          meta_tiempo = meta_tiempo
          sheet_totales['B3'] = "Te faltan: "+str(meta_tiempo)+" Horas"
    else:
          meta_tiempo = meta_tiempo - 1
          sheet_totales['B3'] = "Te faltan: "+str(meta_tiempo)+" Horas y "+str(minutos_faltantes)+" Minutos"
  if meta_tiempo == 1:
      if minutos_faltantes == 0:
           sheet_totales['B3'] = "Te falta una Hora"
      else: 
          
           sheet_totales['B3'] = "Te falta una Hora y "+str(minutos_faltantes)+" minutos."

  if meta_tiempo == 0:
    if minutos_faltantes == 0: 
           sheet_totales['B3'] ="Completaste tu Meta" 

    else:   
           sheet_totales['B3'] = "Te faltan: "+str(minutos_faltantes)+" Minutos"
      
  if meta_tiempo < 0:
  
           sheet_totales['B3'] ="Completaste tu Meta" 

      
# Guarda el libro de trabajo con el nombre deseado
workbook.save('/content/drive/MyDrive/informes_predicacion/informe_predicacion_marzo.xlsx')



# Carga las dos hojas de cálculo en DataFrames de Pandas
df1 = pd.read_excel('/content/drive/MyDrive/informes_predicacion/informe_predicacion_marzo.xlsx', sheet_name='Registros')
df2 = pd.read_excel('/content/drive/MyDrive/informes_predicacion/informe_predicacion_marzo.xlsx', sheet_name='Totales')
display(Markdown("**Totales**"))
# Muestra el primer DataFrame en una tabla interactiva en Colab
display(data_table.DataTable(df2))

print("")
print("")
display(Markdown("**Registros**"))
# Muestra el segundo DataFrame en otra tabla interactiva debajo de la primera
display(data_table.DataTable(df1))

print("")
print("")

# Definimos la función que se ejecutará al hacer clic en el botón "Cerrar"
def on_cerrar_clicked(b):
# Borramos la visualización del DataFrame del archivo de Excel
  from IPython.display import clear_output
  clear_output()


# Creamos el botón "Cerrar"
cerrar_button = Button(description="Cerrar")

# Creamos un contenedor HBox para mostrar los botones en una fila
botones = HBox([cerrar_button])

# Asignamos la función on_cerrar_clicked al evento "click" del botón "Cerrar"
cerrar_button.on_click(on_cerrar_clicked)
display(botones)

In [ ]:
#@title Eliminar Fila completa de "Registros" (segun index)
#@markdown El index parte de 0 fijarse bien en "Ver Excel"
import openpyxl
import ipywidgets as widgets
from IPython.display import clear_output
# Abrir el archivo de Excel
workbook = openpyxl.load_workbook('/content/drive/MyDrive/informes_predicacion/informe_predicacion_marzo.xlsx')

# Seleccionar la hoja de trabajo
sheet = workbook.active

# Crea una caja de texto para ingresar el número de fila a eliminar
index_eliminado = widgets.Text(placeholder='Ingresa el Index a eliminar')

# Crea un botón para eliminar la fila
eliminar_fila_button = widgets.Button(description='Eliminar')
hola = False
# Define la función que se ejecutará al hacer clic en el botón
def eliminar_fila(b):
  global hola
  # Obtén el valor ingresado en la caja de texto
  fila_a_eliminar = int(index_eliminado.value) + 2
  # Elimina la fila

  sheet.delete_rows(fila_a_eliminar, 1)
  # Guarda los cambios en el archivo de Excel
  workbook.save('/content/drive/MyDrive/informes_predicacion/informe_predicacion_marzo.xlsx')
  hola = True
  if hola:
    clear_output()
    # Importamos los widgets que necesitamos
    from ipywidgets import Button, HBox
    from google.colab import data_table
    data_table.enable_dataframe_formatter()
    # Creamos el botón "Cerrar"
    cerrar_button = Button(description="Cerrar")
    # Creamos un contenedor HBox para mostrar los botones en una fila
    botones = HBox([cerrar_button])
    # Ahora podemos acceder al archivo de Excel en Google Drive
    path = '/content/drive/MyDrive/informes_predicacion/informe_predicacion_marzo.xlsx'

    # Podemos usar la biblioteca pandas para leer el archivo de Excel
    import pandas as pd
    df = pd.read_excel(path)

    # Mostramos el DataFrame del archivo de Excel
    display(df)
          


    # Definimos la función que se ejecutará al hacer clic en el botón "Cerrar"

    def on_cerrar_clicked(b):
    # Borramos la visualización del DataFrame del archivo de Excel
      from IPython.display import clear_output
      clear_output()


    # Asignamos la función on_cerrar_clicked al evento "click" del botón "Cerrar"
    cerrar_button.on_click(on_cerrar_clicked)



        # Mostramos el contenedor en la pantalla
    display(botones)



# Asigna la función al evento on_click del botón
eliminar_fila_button.on_click(eliminar_fila)

# Coloca la caja de texto y el botón en una fila horizontal
widgets.HBox([index_eliminado, eliminar_fila_button])

In [ ]:
#@title Informe Febrero
from ipywidgets import Button, HBox
from IPython.display import clear_output, Markdown
import pandas as pd
from google.colab import data_table
workbook = openpyxl.load_workbook('/content/drive/MyDrive/informes_predicacion/informe_predicacion_marzo.xlsx')
sheet = workbook["Registros"]

# Inicializar la variable suma
suma_horas = 0
suma_minutos = 0
total_minutos = 0
total_horas_xd = 0
Total_Publicaciones = 0
Total_Videos  = 0
Total_Revisitas = 0

# Iterar a través de las filas y sumar los valores en la columna C
for row in sheet.iter_rows(min_row=2, max_row=81, min_col=3, max_col=3):
  for cell in row:
    if cell.value is not None:
      suma_horas += cell.value
    else:
       suma_horas += 0


# Iterar a través de las filas y sumar los valores en la columna C
for row in sheet.iter_rows(min_row=2, max_row=81, min_col=4, max_col=4):
  for cell in row:
    if cell.value is not None:
      suma_minutos += cell.value
    else:
      suma_minutos += 0

# Iterar a través de las filas y sumar los valores en la columna C
for row in sheet.iter_rows(min_row=2, max_row=81, min_col=5, max_col=5):
  for cell in row:
    if cell.value is not None:
      Total_Publicaciones += cell.value
    else:
      Total_Publicaciones += 0

# Iterar a través de las filas y sumar los valores en la columna C
for row in sheet.iter_rows(min_row=2, max_row=81, min_col=6, max_col=6):
  for cell in row:
    if cell.value is not None:
      Total_Videos += cell.value
    else:
      Total_Videos += 0

# Iterar a través de las filas y sumar los valores en la columna C
for row in sheet.iter_rows(min_row=2, max_row=81, min_col=7, max_col=7):
  for cell in row:
    if cell.value is not None:
      Total_Revisitas += cell.value
    else:
      Total_Revisitas += 0                    



total_horas_xd = suma_horas + int(suma_minutos/60)
total_minutos = suma_minutos%60

#Abrir hoja totales
sheet_totales = workbook["Totales"]
sheet_totales['A2'] = total_horas_xd
sheet_totales['B2'] = total_minutos
sheet_totales['C2'] = Total_Publicaciones
sheet_totales['D2'] = Total_Videos
sheet_totales['E2'] = Total_Revisitas

# Leer el contenido de la celda A3 y guardar solo las últimas dos letras
meta_horas = sheet_totales['A3'].value
if meta_horas is not None:
    meta_horas = int(meta_horas[-2:])
    
else:
    meta_horas = 0


if meta_horas > 0:
  meta_tiempo = meta_horas
  #meta_tiempo = 25
  meta_tiempo = meta_tiempo-total_horas_xd
  #meta_tiempo = 25 - 24

  
  if total_minutos != 0:
        minutos_faltantes= 60-total_minutos
        if  meta_tiempo == 1 and minutos_faltantes >0:
            meta_tiempo = meta_tiempo - 1
  else:
          minutos_faltantes=0
          if  meta_tiempo == 1 and minutos_faltantes >0:
            meta_tiempo = meta_tiempo - 1
  if meta_tiempo > 1:
    if minutos_faltantes == 0:
          meta_tiempo = meta_tiempo - 1
          sheet_totales['B3'] = "Te faltan: "+str(meta_tiempo)+" Horas"
    else:
          meta_tiempo = meta_tiempo - 1
          sheet_totales['B3'] = "Te faltan: "+str(meta_tiempo)+" Horas y "+str(minutos_faltantes)+" Minutos"
  if meta_tiempo == 1:
      if minutos_faltantes == 0:
           sheet_totales['B3'] = "Te falta una Hora"
      else: 
          
           sheet_totales['B3'] = "Te falta una Hora y "+str(minutos_faltantes)+" minutos."

  if meta_tiempo == 0:
    if minutos_faltantes == 0: 
           sheet_totales['B3'] ="Completaste tu Meta" 

    else:   
           sheet_totales['B3'] = "Te faltan: "+str(minutos_faltantes)+" Minutos"
      
  if meta_tiempo < 0:
  
           sheet_totales['B3'] ="Completaste tu Meta" 

      
# Guarda el libro de trabajo con el nombre deseado
workbook.save('/content/drive/MyDrive/informes_predicacion/informe_predicacion_marzo.xlsx')

from IPython.display import display, HTML, Markdown


text = "Mi Informe de Marzo es:\n\n"

if total_horas_xd != 0:
  text += "• " + str(total_horas_xd) + " Horas\n\n"

if Total_Publicaciones != 0:
  text += "• " + str(Total_Publicaciones) + " Publicaciones\n\n"

if Total_Videos != 0:
  text += "• " + str(Total_Videos) + " Videos\n\n"

if Total_Revisitas != 0:
  text += "• " + str(Total_Revisitas) + " Revisitas"

display(Markdown(text))




# Crea un botón "Copiar"
button = HTML(f'<button onclick="copyAndDelete(this, {text!r})">Copiar</button>')
display(button)

# Define la función `copyAndDelete`
javascript = """
<script>
function copyAndDelete(button, text) {
  var input = document.createElement('textarea');
  document.body.appendChild(input);
  input.value = text;
  input.focus();
  input.select();
  document.execCommand('Copy');
  input.remove();
  button.remove();
  var message = document.createElement('div');
  message.innerHTML = 'Mensaje copiado en Portapapeles!!';
  document.body.appendChild(message);
}
</script>
"""
display(HTML(javascript))

if total_minutos != 0:
  print("----------------------------------------------------------------------------")
  print("Recuerda que te sobraron "+ str(total_minutos)+" minutos en este mes! (No se copiará este mensaje)")
  print("----------------------------------------------------------------------------")